Easy

In [1]:
import os
import re
import random
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

/home/outcast/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
base_path = '../data/cars'
toyota_path = '../data/cars/toyota.csv'
numeric_cols = ['mileage', 'tax', 'mpg', 'engineSize']

In [3]:
data = pd.read_csv(toyota_path)
x_train, x_test, y_train, y_test = train_test_split(
    data[numeric_cols],
    data["price"],
    train_size=0.75,
    random_state=42
)

model = DecisionTreeRegressor()
model.fit(x_train, y_train)

y_test_pred = model.predict(x_test)
print("r2 score ", r2_score(y_test_pred, y_test))

r2 score  0.8925011370124591


In [4]:
print(model.feature_importances_)

[0.2764145  0.02159863 0.08848557 0.61350131]


engineSize - самый важный признак, в целом логично, так как размер двигателя очень сильно влияет на цену автомобиля, как и пробег

Medium


In [5]:
data = pd.DataFrame()
for file in os.listdir(base_path):
    temp = pd.read_csv(os.path.join(base_path, file))
    if data.empty:
        data = temp
    else:
        data = pd.concat([data, temp])

In [6]:
data = data[data['price'].isna() == False]

In [7]:
categorial_cols = ['model', 'transmission', 'fuelType']
data_x = data[categorial_cols]
data_y = data['price']
data_x = data_x.apply(LabelEncoder().fit_transform)

In [8]:
data_y = data_y.apply(lambda x: float(re.sub("[^0-9]", "", str(x))))

In [9]:
x_train, x_test, y_train, y_test = train_test_split(
    data_x,
    data_y,
    train_size=0.75,
    random_state=42
)

In [10]:
from sklearn.model_selection import GridSearchCV

params = {
    "criterion": ('friedman_mse', 'squared_error', 'poisson', 'absolute_error'),
    "max_depth": list(range(3, 10)),
}
search = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    params,
)

In [11]:
search.fit(x_train, y_train)

GridSearchCV(estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'criterion': ('friedman_mse', 'squared_error',
                                       'poisson', 'absolute_error'),
                         'max_depth': [3, 4, 5, 6, 7, 8, 9]})

In [12]:
params, score = search.best_params_, search.best_score_
print(params, score)

{'criterion': 'poisson', 'max_depth': 9} 0.598519825135523


Видно, что результат стал сильно хуже при работе со всей выборкой, видимо для больших данных, которые еще к тому же не относятся к одной марке автомобилей лучше попробовать другой алгоритм